# Inventor Profiles
In this notebook, the we want to look at the behaviours of citation etc. from an inventor perspective. Doing so, we want to know and understand the nature of inventors' profiles for extremely successful patents look like. This notebook is divided to the following sections:
1. Getting inventors of successful profiles
1. Get all patents for the successful inventors
1. Create portfolio **sequential** patents and their behaviours
1. Create portfolio stats

In [248]:
import neo4j
import pandas as pd
import random
import numpy as np
import datetime
from credentials import uri, user, pwd
from patent_neo4j.connection import Neo4jConnection
from functools import reduce

## Get Inventors of Successful Patents
Using the important patents dataset, we obtain the set of successful inventors

In [163]:
df = pd.read_csv("./Data/important_patents.csv")

In [164]:
df.head()

,id,name
0,4136359,AppleMicrocomputer
1,4237224,MolecularChimeras
2,4371752,DigitalVoiceMailSystems
3,4399216,Co-transformationGeneCoding
4,4683195,PolymeraseChainReaction


In [165]:
roots = df['id']

In [166]:
conn = Neo4jConnection(uri, user, pwd)
patent_info = pd.DataFrame(columns=['id', 'date', 'country', 'claims', 'kind', 'county_fips', 'city','state', 'organization', 'org_type', 'inventor_id', 'attribution_stat'])

In [167]:
for root in roots:
    patent_info = pd.concat([patent_info, conn.query_patent_info(root)])

In [168]:
inventor_set = set(patent_info['inventor_id'])

### Randomly Selecting for Inventors with 10 to 30 Patents

In [249]:
df = pd.read_csv("./Data/ten_to_thirty_patents.csv")

In [250]:
df = df.sample(1000)

In [251]:
inventor_set = set(df['inventor'])

### Top 1 percent Number of Citations 

In [296]:
df = pd.read_csv("./Data/top_one_percent_citations.csv")
conn = Neo4jConnection(uri, user, pwd)
patent_info = pd.DataFrame(columns=['id', 'date', 'country', 'claims', 'kind', 'county_fips', 'city','state', 'organization', 'org_type', 'inventor_id', 'attribution_stat'])

In [297]:
df.head()

,Unnamed: 0,patent_id,num_citations
0,1,3953566,1086
1,2,3971065,909
2,3,4092732,1077
3,4,4100324,1051
4,5,4179337,1464


In [298]:
roots = df['patent_id']
for root in roots:
    patent_info = pd.concat([patent_info, conn.query_patent_info(root)])

In [299]:
inventor_set = set(patent_info['inventor_id'])

In [300]:
len(inventor_set)

1785

## Patents of Successful Inventors
In this section, we query all of the patents created by the successful inventors. This would allow us to then construct a **portfolio** for the inventors and study the behaviours of the innovation market.

In [301]:
df_patents = pd.DataFrame(columns = ['inventor_id', 'patent_id', 'patent_date'])

In [302]:
for inventor in inventor_set:
    df_patents = pd.concat([df_patents, conn.query_inventor_profile(inventor)])

In [303]:
df_patents.head()

,inventor_id,patent_id,patent_date
0,fl:p_ln:christenson-4,4361730,1982-11-30
1,fl:p_ln:christenson-4,5279294,1994-01-18
2,fl:p_ln:christenson-4,7706747,2010-04-27
3,fl:p_ln:christenson-4,6123577,2000-09-26
0,fl:s_ln:blumenau-2,7860986,2010-12-28


In [304]:
df_patents.groupby(['inventor_id']).count()

,patent_id,patent_date
inventor_id,,
151eh25cg6vxrbv0ws37ap73w,1,1
2v8lf38ec7i7ju6yoh1m2r3w1,1,1
3b87n45c393xbs20rzr009jyr,1,1
4fc74yy2t1l1yxaupwzifh0zk,1,1
6axve4k099srymqrbtra9fomd,1,1
...,...,...
w9v2cvd89srv2arbe8p6mpsuo,1,1
xlp5zfjw0wj2ickdtcuopjqbq,1,1
y2wj8xpkyax4w4c8guha36bkb,1,1


## Sequential Patents
Convert dates of patents into UNIX timestamp to allow for the sorting of the data. 

In [305]:
import datetime

In [306]:
df_patents = df_patents.dropna()

In [307]:
df_patents['patent_date'] = df_patents['patent_date'].astype(str).apply(datetime.datetime.strptime, args=("%Y-%m-%d",))

In [308]:
df_patents['patent_date'] = df_patents['patent_date'].apply(datetime.datetime.timestamp)

In [309]:
df_patents.head()

,inventor_id,patent_id,patent_date
0,fl:p_ln:christenson-4,4361730,4.074840e+08
1,fl:p_ln:christenson-4,5279294,7.588728e+08
2,fl:p_ln:christenson-4,7706747,1.272344e+09
3,fl:p_ln:christenson-4,6123577,9.699444e+08
0,fl:s_ln:blumenau-2,7860986,1.293516e+09


In [310]:
patent_list = list(set(df_patents['patent_id']))

In [311]:
len(patent_list)

47764

## Patent Statistics
Obtain patent level statistics for each of the patent that we have. This would allow the understanding of how the patents of specific inventors behave 

In [312]:
patent_info = conn.query_batch_patent_info(patent_list)

In [313]:
# patent_info returns a list of patent information 
# this line combines them
df_info = reduce(lambda left,right: pd.merge(left,right,on=['patent_id'],
                                            how='outer'), patent_info)

In [314]:
df_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189834 entries, 0 to 189833
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   patent_id      189834 non-null  object 
 1   num_citations  138038 non-null  float64
 2   organization   186776 non-null  object 
 3   name           185794 non-null  object 
 4   org_type       186292 non-null  object 
 5   inventor_id    189834 non-null  object 
 6   county_fips    130471 non-null  object 
 7   city           183145 non-null  object 
 8   state          146503 non-null  object 
dtypes: float64(1), object(8)
memory usage: 14.5+ MB


In [315]:
# Fill the number of citations that are NA to 0
df_info['num_citations'] = df_info['num_citations'].fillna(0)

In [316]:
df_info = df_info.drop_duplicates()

In [317]:
df_info = df_info.groupby('patent_id').agg(lambda x: set(x)).reset_index()

In [318]:
df_info = df_info.applymap(lambda x: x.pop() if (len(x) == 1) else x)

In [319]:
df_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47764 entries, 0 to 47763
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   patent_id      47764 non-null  object 
 1   num_citations  47764 non-null  float64
 2   organization   46264 non-null  object 
 3   name           46082 non-null  object 
 4   org_type       46135 non-null  object 
 5   inventor_id    47764 non-null  object 
 6   county_fips    33453 non-null  object 
 7   city           45193 non-null  object 
 8   state          37586 non-null  object 
dtypes: float64(1), object(8)
memory usage: 3.3+ MB


In [320]:
df_info.head()

,patent_id,num_citations,organization,name,org_type,inventor_id,county_fips,city,state
0,10000714,0.0,4f322ef0-cdb2-4709-913f-eee9c00cbdab,Cerion LLC,2.0,"{fl:a_ln:difrancesco-1, fl:r_ln:hailstone-1, f...",36055.0,Rochester,NY
1,10001450,0.0,1fcf17fc-565e-45fd-92cc-e5399660c4f5,"MEDTRONIC MINIMED, INC.",2.0,"{fl:b_ln:grosman-1, fl:j_ln:mastrototaro-1, fl...",None,Northridge,CA
2,10001817,15.0,67ab24af-95db-477c-b444-1d81ecab0c54,Apple Inc.,2.0,"{fl:d_ln:kerr-1, fl:c_ln:karunamuni-2, fl:a_ln...",6085.0,Cupertino,CA
3,10002121,0.0,67ab24af-95db-477c-b444-1d81ecab0c54,Apple Inc.,2.0,"{fl:c_ln:karunamuni-2, fl:g_ln:steele-4, fl:p_...",6085.0,Cupertino,CA
4,10002199,1.0,67ab24af-95db-477c-b444-1d81ecab0c54,Apple Inc.,2.0,"{fl:l_ln:soto matamala-1, fl:r_ln:huang-6, fl:...",6085.0,Cupertino,CA


In [321]:
df_info = df_info.rename(columns={"inventor_id":"inventor_set"})

## Putting Things Together
In order to understand the behavior of inventor post-peak, we take the lead of the inventor and observe if the future inventor set is the same as the past

In [322]:
df = pd.merge(df_patents, df_info, on='patent_id', how='left')

In [323]:
df.head()

,inventor_id,patent_id,patent_date,num_citations,organization,name,org_type,inventor_set,county_fips,city,state
0,fl:p_ln:christenson-4,4361730,4.074840e+08,167.0,d70ec079-b006-47b7-8d94-ab9a2ec9df72,Warner Communications Inc.,2.0,"{fl:p_ln:christenson-4, gwobjm59am226gws58l4a1...",36061.0,New York,NY
1,fl:p_ln:christenson-4,5279294,7.588728e+08,922.0,afe42393-0182-4491-b770-d6e2c747eada,Cascade Corporation,2.0,"{fl:r_ln:jessen-2, fl:p_ln:christenson-4, fl:p...",27053.0,Eden Prairie,MN
2,fl:p_ln:christenson-4,7706747,1.272344e+09,5.0,e7e573ed-869d-437a-b9ec-dc5d86ee3700,"The DIRECTV Group, Inc.",2.0,"{fl:p_ln:christenson-4, fl:w_ln:ladrach-1, fl:...",6037.0,El Segundo,CA
3,fl:p_ln:christenson-4,6123577,9.699444e+08,82.0,5fd2bfaa-f87a-4e2f-a049-842eddf06d99,"Energy Transformation Systems, Inc.",2.0,"{fl:p_ln:christenson-4, fl:d_ln:contois-3, fl:...",6081.0,Menlo Park,CA
4,fl:s_ln:blumenau-2,7860986,1.293516e+09,4.0,36bbeb70-9f21-4de3-8f53-01b9fed11e90,EMC IP Holding Company LLC,2.0,"{fl:c_ln:hackett-5, fl:m_ln:derrico-2, fl:s_ln...",25017.0,Hopkinton,MA


In [324]:
# Evaluate number of inventors in the team as well as the assignees of the patent
df['team_size'] = df['inventor_set'].apply(lambda x: len(x) if type(x) is set else 1)
df['num_assignees'] = df['organization'].apply(lambda x: len(x) if type(x) is set else 1)

In [325]:
# Sort values of inventor and patent date and create "lead" on columns for organization and inventor set
df = df.sort_values(by=["inventor_id", "patent_date"], ascending=[True, True])
shifted = df[['inventor_id', "organization", "inventor_set"]].groupby("inventor_id").shift(1)
df = df.join(shifted.rename(columns=lambda x: x+"_lead"))

In [326]:
# Evaluate if the current inventor set is the same as the previous inventor set
df["same_prev_team"] = (df["inventor_set"] == df["inventor_set_lead"]).astype(int)
df["same_prev_assignee"] = (df["organization"] == df["organization_lead"]).astype(int)

In [327]:
df.head()

,inventor_id,patent_id,patent_date,num_citations,organization,name,org_type,inventor_set,county_fips,city,state,team_size,num_assignees,organization_lead,inventor_set_lead,same_prev_team,same_prev_assignee
65611,151eh25cg6vxrbv0ws37ap73w,5179517,7.268184e+08,867.0,d825b74b-dd8e-4345-84ad-43951b87c34a,"Bally Technology, Inc.",2.0,"{fl:w_ln:wellman-7, 151eh25cg6vxrbv0ws37ap73w,...",17031.0,Chicago,IL,4,1,NaN,NaN,0,0
24049,2v8lf38ec7i7ju6yoh1m2r3w1,5710887,8.852760e+08,1594.0,7709b401-8dc1-4e01-922c-2876cb2be714,"BroadVision, Inc.",2.0,"{fl:w_ln:yamamoto-5, fl:j_ln:hempe-2, fl:e_ln:...",6085.0,Los Altos,CA,11,1,NaN,NaN,0,0
20401,3b87n45c393xbs20rzr009jyr,5165407,7.225848e+08,1167.0,7bddc197-41c1-4dad-a4e4-e8f790cefc83,THE UNIVERSITY OF KANSAS,2.0,"{fl:g_ln:wilson-66, fl:b_ln:hill-13, 3b87n45c3...",20045.0,Lawrence,KS,7,1,NaN,NaN,0,0
32701,4fc74yy2t1l1yxaupwzifh0zk,5523520,8.338644e+08,3899.0,2e224e60-7c41-4fa7-aa17-b4560350f966,Goldsmith Seeds Europe B.V.,3.0,"{4fc74yy2t1l1yxaupwzifh0zk, fl:m_ln:holtrop-2}",6085.0,Gilroy,CA,2,1,NaN,NaN,0,0
23459,6axve4k099srymqrbtra9fomd,6676660,1.073974e+09,918.0,d463aba3-a1e4-4301-ab42-1d9d6fc3a82e,"Ethicon Endo-Surgery, Inc.",2.0,"{fl:j_ln:vaitekunas-1, fl:s_ln:wampler-1, 6axv...",39061.0,Cincinnati,OH,6,1,NaN,NaN,0,0


In [328]:
columns = ["inventor_id","patent_id","patent_date","num_citations","team_size","num_assignees","same_prev_assignee","same_prev_team"]
df[columns].to_csv("inventor_profiles.csv", index=False)

In [329]:
df.head()

,inventor_id,patent_id,patent_date,num_citations,organization,name,org_type,inventor_set,county_fips,city,state,team_size,num_assignees,organization_lead,inventor_set_lead,same_prev_team,same_prev_assignee
65611,151eh25cg6vxrbv0ws37ap73w,5179517,7.268184e+08,867.0,d825b74b-dd8e-4345-84ad-43951b87c34a,"Bally Technology, Inc.",2.0,"{fl:w_ln:wellman-7, 151eh25cg6vxrbv0ws37ap73w,...",17031.0,Chicago,IL,4,1,NaN,NaN,0,0
24049,2v8lf38ec7i7ju6yoh1m2r3w1,5710887,8.852760e+08,1594.0,7709b401-8dc1-4e01-922c-2876cb2be714,"BroadVision, Inc.",2.0,"{fl:w_ln:yamamoto-5, fl:j_ln:hempe-2, fl:e_ln:...",6085.0,Los Altos,CA,11,1,NaN,NaN,0,0
20401,3b87n45c393xbs20rzr009jyr,5165407,7.225848e+08,1167.0,7bddc197-41c1-4dad-a4e4-e8f790cefc83,THE UNIVERSITY OF KANSAS,2.0,"{fl:g_ln:wilson-66, fl:b_ln:hill-13, 3b87n45c3...",20045.0,Lawrence,KS,7,1,NaN,NaN,0,0
32701,4fc74yy2t1l1yxaupwzifh0zk,5523520,8.338644e+08,3899.0,2e224e60-7c41-4fa7-aa17-b4560350f966,Goldsmith Seeds Europe B.V.,3.0,"{4fc74yy2t1l1yxaupwzifh0zk, fl:m_ln:holtrop-2}",6085.0,Gilroy,CA,2,1,NaN,NaN,0,0
23459,6axve4k099srymqrbtra9fomd,6676660,1.073974e+09,918.0,d463aba3-a1e4-4301-ab42-1d9d6fc3a82e,"Ethicon Endo-Surgery, Inc.",2.0,"{fl:j_ln:vaitekunas-1, fl:s_ln:wampler-1, 6axv...",39061.0,Cincinnati,OH,6,1,NaN,NaN,0,0


## Inventors and Assignees Over Time
To further investigate the relationship between inventors and assignees, we want to see the *cumulative* inventors and assignees over time.

We introduce a variable called **cumulative inventor** which is the set of all inventors the inventor has worked with up to that point.

In [330]:
# Make sure there is no NA and that all of them are sets
df['inventor_set'] = df['inventor_set'].fillna('0')
df['inventor_set'] = df['inventor_set'].apply(lambda x: set([x]) if (type(x) is not set) else x)

In [331]:
# For every inventor
    # For each patent
        # Add individual inventor set
cumulative_inventor = []
for inventor in inventor_set:
    cumulative_set = set()
    patents = df[df['inventor_id'] == inventor]['patent_id']
    for patent in patents:
        cumulative_set = cumulative_set.union(df[(df['inventor_id'] == inventor) & (df['patent_id'] == patent)].inventor_set.tolist()[0])
        cumulative_inventor.append([inventor, patent, cumulative_set])
        

In [332]:
# Create dataframe for the sets and create counts
cumulative_inv = pd.DataFrame(cumulative_inventor, columns = ['inventor_id', 'patent_id', 'cumulative_inventor'])
cumulative_inv['count_cumulative_inv'] = cumulative_inv['cumulative_inventor'].apply(lambda x: len(x))

In [333]:
# Do the same for organization
df['organization'] = df['organization'].fillna('0')
df['organization'] = df['organization'].apply(lambda x: set([x]) if (type(x) is not set) else x)

In [334]:
# For every inventor
    # For each patent
        # Add individual assignee set
cumulative_assignee = []
for inventor in inventor_set:
    cumulative_set = set()
    patents = df[df['inventor_id'] == inventor]['patent_id']
    for patent in patents:
        cumulative_set = cumulative_set.union(df[(df['inventor_id'] == inventor) & (df['patent_id'] == patent)].organization.tolist()[0])
        cumulative_assignee.append([inventor, patent, cumulative_set])

In [335]:
# Create dataframe for the sets and create counts
cumulative_assignee = pd.DataFrame (cumulative_assignee, columns = ['inventor_id', 'patent_id', 'cumulative_assignee'])
cumulative_assignee['count_cumulative_assignee'] = cumulative_assignee['cumulative_assignee'].apply(lambda x: len(x))

In [336]:
cumulative_inv.head()

,inventor_id,patent_id,cumulative_inventor,count_cumulative_inv
0,fl:p_ln:christenson-4,4361730,"{fl:p_ln:christenson-4, gwobjm59am226gws58l4a1...",8
1,fl:p_ln:christenson-4,5279294,"{fl:p_ln:christenson-4, fl:r_ln:jessen-2, fl:p...",13
2,fl:p_ln:christenson-4,6123577,"{fl:p_ln:christenson-4, fl:r_ln:jessen-2, fl:p...",15
3,fl:p_ln:christenson-4,7706747,"{fl:p_ln:christenson-4, fl:r_ln:jessen-2, fl:t...",19
4,fl:s_ln:blumenau-2,5391993,"{fl:s_ln:blumenau-2, fl:m_ln:khazam-1}",2


In [337]:
cumulative_assignee.head()

,inventor_id,patent_id,cumulative_assignee,count_cumulative_assignee
0,fl:p_ln:christenson-4,4361730,{d70ec079-b006-47b7-8d94-ab9a2ec9df72},1
1,fl:p_ln:christenson-4,5279294,"{afe42393-0182-4491-b770-d6e2c747eada, d70ec07...",2
2,fl:p_ln:christenson-4,6123577,"{afe42393-0182-4491-b770-d6e2c747eada, d70ec07...",3
3,fl:p_ln:christenson-4,7706747,"{afe42393-0182-4491-b770-d6e2c747eada, e7e573e...",4
4,fl:s_ln:blumenau-2,5391993,{97102f28-0d9d-460a-b558-10f2384094ee},1


In [338]:
# Join tables
columns = ['inventor_id', 'patent_id', 'patent_date', 'num_citations', 'organization', 'inventor_set', 'team_size']
df_cumulative = df[columns].merge(cumulative_inv, how='left', on=['inventor_id', 'patent_id'])
df_cumulative = df_cumulative.merge(cumulative_assignee, how='left', on=['inventor_id', 'patent_id'])

In [339]:
df_cumulative.head(5)

,inventor_id,patent_id,patent_date,num_citations,organization,inventor_set,team_size,cumulative_inventor,count_cumulative_inv,cumulative_assignee,count_cumulative_assignee
0,151eh25cg6vxrbv0ws37ap73w,5179517,7.268184e+08,867.0,{d825b74b-dd8e-4345-84ad-43951b87c34a},"{fl:w_ln:wellman-7, 151eh25cg6vxrbv0ws37ap73w,...",4,"{fl:w_ln:wellman-7, 151eh25cg6vxrbv0ws37ap73w,...",4,{d825b74b-dd8e-4345-84ad-43951b87c34a},1
1,2v8lf38ec7i7ju6yoh1m2r3w1,5710887,8.852760e+08,1594.0,{7709b401-8dc1-4e01-922c-2876cb2be714},"{fl:w_ln:yamamoto-5, fl:j_ln:hempe-2, fl:e_ln:...",11,"{fl:w_ln:yamamoto-5, fl:j_ln:hempe-2, fl:e_ln:...",11,{7709b401-8dc1-4e01-922c-2876cb2be714},1
2,3b87n45c393xbs20rzr009jyr,5165407,7.225848e+08,1167.0,{7bddc197-41c1-4dad-a4e4-e8f790cefc83},"{fl:g_ln:wilson-66, fl:b_ln:hill-13, 3b87n45c3...",7,"{fl:g_ln:wilson-66, fl:g_ln:reach-1, fl:d_ln:b...",7,{7bddc197-41c1-4dad-a4e4-e8f790cefc83},1
3,4fc74yy2t1l1yxaupwzifh0zk,5523520,8.338644e+08,3899.0,{2e224e60-7c41-4fa7-aa17-b4560350f966},"{4fc74yy2t1l1yxaupwzifh0zk, fl:m_ln:holtrop-2}",2,"{4fc74yy2t1l1yxaupwzifh0zk, fl:m_ln:holtrop-2}",2,{2e224e60-7c41-4fa7-aa17-b4560350f966},1
4,6axve4k099srymqrbtra9fomd,6676660,1.073974e+09,918.0,{d463aba3-a1e4-4301-ab42-1d9d6fc3a82e},"{fl:j_ln:vaitekunas-1, fl:s_ln:wampler-1, 6axv...",6,"{6axve4k099srymqrbtra9fomd, fl:s_ln:wampler-1,...",6,{d463aba3-a1e4-4301-ab42-1d9d6fc3a82e},1


In [340]:
def new_kid_in_block(a,b):
    # Get the set difference and return the length
    diff = a.difference(b)
    return(len(diff))    

In [341]:
# Get lead of cumulative inventor and assignee
shifted = df_cumulative[['inventor_id', "cumulative_inventor", "cumulative_assignee"]].groupby("inventor_id").shift(1)
df_cumulative = df_cumulative.join(shifted.rename(columns=lambda x: x+"_lead"))

In [342]:
# Change into set() cumulative change
df_cumulative['cumulative_inventor_lead'] = df_cumulative['cumulative_inventor_lead'].fillna(0)
df_cumulative['cumulative_inventor_lead'] = df_cumulative['cumulative_inventor_lead'].apply(lambda x: set() if x == 0 else x)
df_cumulative['cumulative_assignee_lead'] = df_cumulative['cumulative_assignee_lead'].fillna(0)
df_cumulative['cumulative_assignee_lead'] = df_cumulative['cumulative_assignee_lead'].apply(lambda x: set() if x == 0 else x)

In [343]:
# Get the size set difference between cumulative inventor vs current 
df_cumulative['new_inventor'] = df_cumulative.apply(lambda x: new_kid_in_block(x.inventor_set, x.cumulative_inventor_lead), axis=1)
df_cumulative['new_assignee'] = df_cumulative.apply(lambda x: new_kid_in_block(x.organization, x.cumulative_assignee_lead), axis=1)

In [344]:
# Save data
columns = ["inventor_id","patent_id","patent_date","num_citations","team_size","count_cumulative_inv","count_cumulative_assignee","new_inventor","new_assignee"]
df_cumulative[columns].to_csv("inventor_profiles_cumulative.csv", index=False)